In [42]:
import pandas as pd
import numpy as np
import math
import datetime

In [43]:
train_m = pd.read_csv('training_for_final_models.csv')
test_m = pd.read_csv('test_for_final_models.csv')

In [44]:
print list(train_m.columns)

['id', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination', 'sum_secs_elapsed', 'counts', 'pred_lag_account_created', 'pred_lag_first_active', 'pred_bookings']


In [45]:
train_m.head(3)

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,sum_secs_elapsed,counts,pred_lag_account_created,pred_lag_first_active,pred_bookings
0,gxn3p5htnn,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NB,NB,NB
1,820tgsjxq7,MALE,35-39,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,same day,before,early
2,4ft3gnwmtx,FEMALE,55-59,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,NaN,NaN,greater 1 day,greater 1 day,NB


In [119]:
excl = list(train_m.columns)
toremove = ['id', 'country_destination'] 
map(lambda x: excl.remove(x), toremove)

[None, None]

In [120]:
excl

['gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'sum_secs_elapsed',
 'counts',
 'pred_lag_account_created',
 'pred_lag_first_active',
 'pred_bookings']

In [121]:
train_xcl = train_m.loc[:, excl]
test_xcl = test_m.loc[:, excl]

In [122]:
from sklearn.feature_extraction import DictVectorizer

frames = [train_xcl, test_xcl]
all_data = pd.concat(frames)
all_data = all_data.T.to_dict().values()
train_xdic = train_xcl.T.to_dict().values()
test_xdic = test_xcl.T.to_dict().values()

vec = DictVectorizer()
vec.fit(all_data)
train_xvec = vec.transform(train_xdic)
test_xvec = vec.transform(test_xdic)

In [123]:
train_y = train_m.loc[:,'country_destination']

In [124]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_ytrans = le.transform(train_y)
print type(train_ytrans)

<type 'numpy.ndarray'>


In [125]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer


def dcg_score(y_true, y_score, k=5):
    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba= True, k=5)


In [126]:
import numpy as np
import xgboost as xgb
model2 = xgb.XGBClassifier(max_depth = 5, learning_rate = 0.1, n_estimators = 30, silent = False, \
                           objective ="multi:softprob")

In [127]:
model2.fit(train_xvec, train_ytrans, verbose = True, eval_metric = "ndcg@5")

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [128]:
pred2 = model2.predict_proba(test_xvec)

In [129]:
test_id = test_m['id']
 
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(test_id)):
  idx = test_id[i]
  ids += [idx] * 5
  cts += le.inverse_transform(np.argsort(pred2[i])[::-1])[:5].tolist()
 
sub1 = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
print sub1.head
#sub1.to_csv('xgboostonlyaccountcreated.csv',index=False)

<bound method DataFrame.head of                 id country
0       5uwns89zht     NDF
1       5uwns89zht      US
2       5uwns89zht   other
3       5uwns89zht      FR
4       5uwns89zht      GB
5       jtl0dijy2j     NDF
6       jtl0dijy2j      US
7       jtl0dijy2j   other
8       jtl0dijy2j      FR
9       jtl0dijy2j      GB
10      xx0ulgorjt     NDF
11      xx0ulgorjt      US
12      xx0ulgorjt   other
13      xx0ulgorjt      FR
14      xx0ulgorjt      IT
15      6c6puo6ix0     NDF
16      6c6puo6ix0      US
17      6c6puo6ix0   other
18      6c6puo6ix0      FR
19      6c6puo6ix0      IT
20      czqhjk3yfe     NDF
21      czqhjk3yfe      US
22      czqhjk3yfe   other
23      czqhjk3yfe      FR
24      czqhjk3yfe      IT
25      szx28ujmhf     NDF
26      szx28ujmhf      US
27      szx28ujmhf   other
28      szx28ujmhf      FR
29      szx28ujmhf      IT
...            ...     ...
310450  8yvhec201j     NDF
310451  8yvhec201j      US
310452  8yvhec201j   other
310453  8yvhec201j     

In [130]:
pred3 = model2.predict(test_xvec)

In [ ]:
set(le.inverse_transform(pred3))

{'NDF'}

In [ ]:
#Random forest code 
from sklearn import ensemble
import sklearn.grid_search as gs
randomForest = ensemble.RandomForestClassifier(n_jobs = -1, oob_score=True)

trainx_vecmiss = pd.DataFrame(train_xvec.toarray()).fillna(-1)
testx_vecmiss = pd.DataFrame(test_xvec.toarray()).fillna(-1)

np.random.seed(1)
grid_para_forest = [{"n_estimators": [200], "max_features": ['auto']}]
grid_search_forest = gs.GridSearchCV(randomForest, grid_para_forest, scoring=ndcg_scorer, cv=5, n_jobs = -1)
grid_search_forest.fit(trainx_vecmiss, train_ytrans)

In [ ]:
predrf = grid_search_forest.predict_proba(testx_vecmiss)

In [ ]:
test_id = test_m['id']
 
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(test_id)):
  idx = test_id[i]
  ids += [idx] * 5
  cts += le.inverse_transform(np.argsort(predrf[i])[::-1])[:5].tolist()
 
sub1 = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
print sub1.head
sub1.to_csv('rfallpred.csv',index=False)

In [118]:
grid_search_forest.best_score_

0.79449259101871328